# Create `openalex.institutions.raw_affiliation_strings_institutions_mv`

Creates a materialized view that transforms the affiliation strings lookup table by:
1. Applying institution_ids_override when available
2. Filtering out invalid institution_ids: `array(-1)` and arrays containing null values
3. Cleaning up empty country arrays

This materialized view is used by `work_author_affiliations_mv` to join affiliations with institution IDs.

In [ ]:
%sql
CREATE OR REPLACE MATERIALIZED VIEW openalex.institutions.raw_affiliation_strings_institutions_mv AS
SELECT 
  raw_affiliation_string,
  FILTER(
    CASE
      WHEN institution_ids_override != array() THEN institution_ids_override
      WHEN SIZE(institution_ids) > 0 AND institution_ids[0] IS NULL THEN array()
      ELSE COALESCE(institution_ids, array())
    END,
    x -> x IS NOT NULL AND x != -1
  ) AS institution_ids,
  CASE 
    WHEN COALESCE(countries, array()) = array('') THEN array()
    ELSE COALESCE(countries, array())
  END AS countries,
  source,
  institution_ids AS model_institution_ids,
  institution_ids_override,
  created_datetime,
  updated_datetime
FROM openalex.institutions.affiliation_strings_lookup